# Project group 30

In [1]:
#pip install pycountry_convert
#pip install fuzzywuzzy
import numpy as np
import pandas as pd
import pycountry
import pycountry_convert as pcc
from fuzzywuzzy import fuzz, process
pd.options.display.max_rows = 20
np.set_printoptions(precision = 4, suppress = True)

In [3]:
#loading the csv files into dataframes
population = 'population.csv'
df_population= pd.read_csv(population)
medals = 'medals.csv'
df_medals= pd.read_csv(medals)
cpi = 'cpi.csv'
df_cpi= pd.read_csv(cpi)

In [4]:
# Create a list of all unique countries in the dataframe
countries = df_medals['Country'].unique().tolist()

#Create a list of all standard country names
countries_standard = [country.name for country in pycountry.countries]

for i in range(len(df_medals)):
    temp = df_medals['Country'][i]
    country = 'c'
    try:
        country = pycountry.countries.lookup(temp)
    except LookupError as e:
        print('a')
        #continue
    if country == 'c':
        try:
            match = process.extractOne(str(temp), countries_standard, scorer=fuzz.token_sort_ratio)
            if match[1] > 74:
                c = pycountry.countries.get(name=match[0])
                country_code=c.alpha_3
                df_medals.loc[i, 'Country'] = country_code
            else:
                df_medals.loc[i, 'Country'] = None
        except ValueError as e:
            df_medals.loc[i, 'Country'] = None
    else:
        df_medals.loc[i, 'Country'] = country.alpha_3

#manually converting few instances
#Kosovo doesnt have ISO3
df_medals.at[3, 'Country'] = 'GBR'
df_medals.at[4, 'Country'] = 'RUS'
df_medals.at[33, 'Country'] = 'TWN'

/opt/anaconda3/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


a
a
a
a
a
a


In [5]:
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    print(df_medals)

# Methodology

Medals scoring system:
    
    GOLD - 3 points
    SILVER - 2 points
    BRONZE - 1 point


In [7]:
#calculating the value of the medals earned
medal_values=[]
for i in range(len(df_medals)):
    temp_g=df_medals['Gold Medal'][i]
    #print(temp_g)
    temp_s=df_medals['Silver Medal'][i]
    temp_b=df_medals['Bronze Medal'][i]
    value=(temp_g*3)+(temp_s*2)+temp_b
    medal_values.append(value)
df_medals["Medal Value"] = medal_values

In [11]:
#merging medal df with population df
df_merged1 = pd.merge(df_medals, df_population, left_on='Country', right_on='CCA3')

#calculation medal value per million inhabitants
medals_per_million=[]
for i in range(len(df_merged1)):
    medal_adjusted = df_merged1['Medal Value'][i]/(df_merged1['2020 Population'][i]/1000000) 
    medals_per_million.append(medal_adjusted)
df_merged1["Medals per million"] = medals_per_million

#merging the resulting df with cpi df
df_merged2 = pd.merge(df_merged1, df_cpi, left_on='Country', right_on='ISO3')
df_merged2
df = df_merged2[['Country_x','Medal Value', 'Medals per million','2020 Population', 'CPI score 2019']]
df



,Country_x,Medal Value,Medals per million,2020 Population,CPI score 2019
0,USA,232,0.690595,335942003,69
1,CHN,196,0.137551,1424929781,41
2,JPN,126,1.006030,125244761,73
3,GBR,130,1.938578,67059474,77
4,RUS,139,0.954557,145617329,28
...,...,...,...,...,...
83,GHA,1,0.031075,32180401,41
84,GRD,1,8.086493,123663,53
85,KWT,1,0.229334,4360444,40
86,MDA,1,0.324165,3084847,32
